In [2]:
source("./Functions/MOCL.R") 
source("./Functions/lambda_selection.R")
source("./Functions/MOCL_wcss.R")
source("./Functions/Adaptive_MAM.R") 

source("./Functions/SKM_gap.R")
source("./Functions/evaluation.R")


library(VarSelLCM) # VarselLCM
library(cluster)     # PAM

library(ggplot2)
library(gridExtra)
library(ggpubr)
library(ggthemes)
library(GGally)
library(RColorBrewer)
library(corrplot)
library(dplyr)
library(pdfCluster)
library(fossil)
options(warn=-1)
library(Rtsne)

# Function

In [3]:
model=MOCL_wcss

In [4]:
# number of K
cor_sta_sub<-function(X,k,sub_rat,seed1=1,seed2=20,num.knotsf=6,max.iter=500){
    kk=k
    
    set.seed(seed1);sub_1=sample(1:nrow(X),nrow(X)*sub_rat)
    set.seed(seed2);sub_2=sample(1:nrow(X),nrow(X)*sub_rat)

    x_1=X[sub_1,]
    x_2=X[sub_2,]

    or_m1=model(Xf=x_1,k=kk,lambda_seq=seq.default(from=0.001,to=2,length=100),delta=0.01) # knot : 6 -> L: 8
    or_m2=model(Xf=x_2,k=kk,lambda_seq=seq.default(from=0.001,to=2,length=100),delta=0.01)
 
    clu_mat=matrix(0,nrow(X),2)
    clu_mat[sub_1,1]=or_m1$cluster
    clu_mat[sub_2,2]=or_m2$cluster
    c_sub=clu_mat[intersect(sub_1,sub_2),] # similarity of intersected subsample
  
    similarity=abs(cor(c_sub[,1],c_sub[,2],method="kendall"))
  
    return(list(corr=similarity,sub1=sub_1,sub2=sub_2,clu_mat=clu_mat))
}

best_k<-function(X,iter_sub=100,K_list=c(3,4,5,6,7),sub_rat=0.8,max.iter=500){
    k_list=K_list
    iter_sub=iter_sub
    data=X
    
    cor_mat=matrix(0,iter_sub,length(k_list))
    colnames(cor_mat)=paste("K",k_list,sep="_")
    for(i in 1:iter_sub){
        for(kk in 1:length(k_list)){
            set_s=round((as.numeric(Sys.time())*10000))
            set_s=as.numeric(set_s%%1000)
            set.seed(set_s)
            ss=round(runif(10,1,10000))
            
            co_f=cor_sta_sub(X=data,k=k_list[kk],sub_rat=sub_rat,seed1=ss[1],seed2=ss[3],
                                 num.knotsf=6,max.iter=500)
            cor_mat[i,kk]=cor_mat[i,kk]+co_f$corr
        
        }
    
    }
    return(cor_mat)
    
}

# Data

In [5]:
mlb2021=read.csv("/Users/anseongbin/Desktop/AAAI 2024/Supplementary Material for Monotone Clustering/Real Data Analysis/stats2021.csv")
mlb2021 = mlb2021[,-c(1,2,3,4,16)]

colnames(mlb2021)=c("strikeout","walk","batting","power","woba","velocity",
                 "angle","well-hit","barrel","hard","contact")
mlb_x = scale(mlb2021[,-c(5,9)])
y = mlb2021[,5]
colnames(mlb_x)=c("strikeout","walk","batting","power","velocity",
                 "angle","well-hit","hard","contact")

#mlb_x=mlb_x[,c(4,8,9,10)]
colnames(mlb_x)

[1] "strikeout" "walk"      "batting"   "power"     "velocity"  "angle"    
[7] "well-hit"  "hard"      "contact"

In [12]:
Sys.time()
K_stab=best_k(mlb_x,iter_sub=10,K_list=c(3,4,5,6),sub_rat=0.8)
Sys.time()

[1] "2024-01-31 16:15:36 KST"

[1] "2024-01-31 16:22:54 KST"

In [13]:
apply(K_stab,2,mean)

K_3       K_4       K_5       K_6 
0.8551064 0.8812402 0.9280499 0.8995625

In [15]:
2*round(apply(K_stab,2,sd),4)/sqrt(10)

K_3        K_4        K_5        K_6 
0.06084222 0.03604997 0.02675287 0.06488994